In [1]:
%cd ..

/home/alta/Conversational/OET/al826/2022/shortcuts/OOD_generalisation


In [16]:
### TEMP for debugging- delete cell 
import re
import numpy as np
from collections import defaultdict
from typing import List, Tuple

from src.utils.data_utils import load_data
from src.system_loader import EnsembleLoader, SystemLoader
from src.utils.evaluation import get_seed_accuracy, get_accuracy

def get_word_counts(data:List[list])->dict:
    """ returns counts of words in a data set"""
    word_counts = defaultdict(lambda:0)
    for ex in data:
        text = ex['text'].lower()
        words = re.findall(r"[\w']+|[.,!?;]", text)
        word_set = set(words)
        for word in word_set:
            word_counts[word] += 1
    return word_counts

def get_vocab(word_counts:dict, thresh:int=100)->dict:
    """ truncates vocab to those with above thresh occurences"""
    vocab = {word:count for word, count in word_counts.items() if count > thresh}
    return vocab 

def get_vocab_occurences(data, vocab:dict)->list:
    """ returns the ids of all examples which contains the word"""
    word_occurences = {k:[] for k in vocab}
    for k, ex in enumerate(data):
        for word in vocab:
            if word in ex['text'].lower():
                word_occurences[word].append(k)
    return word_occurences

def word_bias(data, vocab, num_labels:int=2)->list:
    """ returns occurences of each word in vocab"""
    vocab = set([k for k in vocab])
    vocab_counts = defaultdict(lambda: np.zeros(num_labels))

    for ex in data:
        text = ex['text'].lower()
        words = re.findall(r"[\w']+|[.,!?;]", text)
        label = ex['label']
        
        for word in set(words).intersection(vocab):
            vocab_counts[word][label] += 1
    return vocab_counts

def word_shortcut_plot(model_path, data_name_1, data_name_2):
    system = EnsembleLoader(model_path) 
    data_preds_1 = system.load_seed_preds(data_name_1, mode='dev')
    data_preds_2 = SystemLoader.load_labels(data_name_1, mode='dev')
    preds = 
    
    split_ids = {'train':0, 'dev':1, 'test':2}
    data = load_data('rt')[split_ids['train']]
word_counts = get_word_counts(data)
vocab = get_vocab(word_counts)
x = get_vocab_occurences(data, vocab)

###################
rt
###################


Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/home/alta/Conversational/OET/al826/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from src.helpers import DataLoader
from src.utils.data_utils import load_data

data_name = 'rt'
split = 'train'
split_ids = {'train':0, 'dev':1, 'test':2}
data = load_data(data_name)[split_ids[split]]
    

###################
rt
###################


Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/home/alta/Conversational/OET/al826/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print(data[0])

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}


In [30]:
import re
from collections import defaultdict, Counter
from typing import List, Tuple

from src.helpers import DataLoader
from src.utils.data_utils import load_data


def flatten(x):
    return [i for j in x for i in j]

def get_biased_words(data_name, split='train', v_thresh=0.01)->List[Tuple['word', 'P(y|w)', 'count']]:
    """ only works for binary label data sets """
    split_ids = {'train':0, 'dev':1, 'test':2}
    data = load_data(data_name)[split_ids[split]]
    
    words_counts = defaultdict(list)
    for ex in data:
        text = ex['text'].lower()
        words = re.findall(r"[\w']+|[.,!?;]", text)
        word_set = set(words)
        label = ex['label']
        words_counts[label] += [list(word_set)]

    pos_words = flatten(words_counts[1])
    neg_words = flatten(words_counts[0])

    pos_counter = Counter(pos_words)
    neg_counter = Counter(neg_words)
    all_counter = Counter(pos_words + neg_words)
    w_freq = {k:v/len(data) for k, v in all_counter.items()}
    
    vocab = [word for word, freq in w_freq.items() if freq > v_thresh]
    word_bias_scores = [(word, pos_counter[word]/all_counter[word], all_counter[word], 100*w_freq[word]) for word in vocab]
    word_bias_scores = [(x[0], round(x[1], 3), x[2], round(x[3], 3)) for x in word_bias_scores]

    word_bias_scores = sorted(word_bias_scores, key=lambda x: x[1], reverse=True)
    return word_bias_scores


In [33]:
bias_words_1 = get_biased_words('imdb', 'train', 0.005)
bias_words_2 = get_biased_words('rt', 'train', 0.003)
print(len(bias_words_1))
print(len(bias_words_2))

###################
imdb
###################


Reusing dataset imdb (/home/alta/Conversational/OET/al826/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

###################
rt
###################


Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/home/alta/Conversational/OET/al826/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

3218
689


In [37]:
print(sorted(bias_words_1, key=lambda x: x[1], reverse=False))

[('incoherent', 0.049, 103, 0.515), ('unfunny', 0.073, 191, 0.955), ('remotely', 0.08, 150, 0.75), ('waste', 0.081, 1043, 5.215), ('atrocious', 0.087, 149, 0.745), ('redeeming', 0.088, 249, 1.245), ('pointless', 0.089, 358, 1.79), ('laughable', 0.09, 321, 1.605), ('worst', 0.097, 1821, 9.105), ('boredom', 0.099, 101, 0.505), ('poorly', 0.1, 500, 2.5), ('lousy', 0.108, 158, 0.79), ('awful', 0.109, 1150, 5.75), ('unconvincing', 0.114, 132, 0.66), ('insult', 0.114, 158, 0.79), ('uninspired', 0.115, 104, 0.52), ('existent', 0.118, 119, 0.595), ('sucks', 0.119, 201, 1.005), ('wasting', 0.12, 108, 0.54), ('stupidity', 0.131, 122, 0.61), ('lame', 0.133, 518, 2.59), ('pathetic', 0.134, 337, 1.685), ('uninteresting', 0.139, 158, 0.79), ('amateurish', 0.142, 176, 0.88), ('crap', 0.144, 702, 3.51), ('badly', 0.148, 459, 2.295), ('dreadful', 0.149, 175, 0.875), ('horrible', 0.15, 800, 4.0), ('wasted', 0.15, 414, 2.07), ('suck', 0.152, 125, 0.625), ('mess', 0.154, 480, 2.4), ('idiotic', 0.155, 116,

In [38]:
print(len(bias_words_1), len(bias_words_2))

3218 689


In [39]:
vocab_1 = [i[0] for i in bias_words_1]
vocab_2 = [i[0] for i in bias_words_2]

overlap_vocab = list(set(vocab_1).intersection(vocab_2))

score_dict_1 = {x[0]:x[1] for x in bias_words_1}
score_dict_2 = {x[0]:x[1] for x in bias_words_2}

bias_diff = [(word, score_dict_1[word] - score_dict_2[word]) for word in overlap_vocab]

for item in (sorted(bias_diff, key=lambda x: abs(x[1]), reverse=True)):
    word = item[0]
    print(word, score_dict_1[word], score_dict_2[word])

tv 0.494 0.083
plays 0.604 0.269
heavy 0.504 0.182
john 0.626 0.306
culture 0.609 0.925
fast 0.454 0.75
falls 0.448 0.156
already 0.445 0.156
future 0.646 0.357
mystery 0.547 0.258
feels 0.509 0.223
honest 0.544 0.828
series 0.583 0.304
too 0.481 0.211
cold 0.582 0.314
title 0.451 0.184
cinema 0.581 0.846
nor 0.426 0.172
help 0.513 0.767
jokes 0.392 0.139
piece 0.426 0.676
political 0.58 0.818
eyes 0.552 0.786
neither 0.371 0.139
leaves 0.539 0.312
camera 0.408 0.633
gone 0.509 0.286
spy 0.692 0.469
next 0.489 0.267
imagine 0.447 0.226
violence 0.531 0.312
overall 0.575 0.357
rich 0.587 0.8
look 0.467 0.68
problem 0.38 0.167
smart 0.568 0.778
formula 0.532 0.323
tries 0.396 0.189
hilarious 0.609 0.816
ends 0.492 0.286
certainly 0.574 0.368
flat 0.27 0.065
gags 0.478 0.273
proves 0.59 0.793
melodrama 0.612 0.41
approach 0.635 0.433
winning 0.723 0.923
things 0.509 0.309
filmmaker 0.478 0.677
thin 0.336 0.138
face 0.492 0.689
b 0.4 0.594
school 0.455 0.263
form 0.584 0.774
women 0.47 0.6

In [ ]:
from collections import Counter


vocab_thresh = 100
vocab = [word for word, count in all_counter.most_common() if count > vocab_thresh]
print(len(vocab))

In [ ]:
print())

In [ ]:
import time
import re
import numpy as np

from collections import Counter, defaultdict
from src.helpers import ConvHandler

### Get training data
C = ConvHandler(None)
train_path = f"data/swda_standard/train.json"
train_data = C.tok_data(train_path)

### Get word label counts for training data
utts_per_label = defaultdict(list)
for conv in train_data:
    for utt in conv:
        text = utt.text.lower()
        words = re.findall(r"[\w']+|[.,!?;]", text)
        label = utt.label_name
        utts_per_label[label] += [words]

### Main Keys for Analysis (later on)
all_labels = [k for k, v in sorted(utts_per_label.items(), key=lambda item: len(item[1]),reverse=True)]
red_labels = all_labels[:9]
label_dist = [len(utts_per_label[k]) for k in all_labels]

### Length Bias

In [ ]:
from types import SimpleNamespace
import matplotlib.pyplot as plt

### Measuring length bias
length_dists = {}
for label, utts in utts_per_label.items():
    lens = [len(utt) for utt in utts]
    
    mean = np.mean(lens)
    std  = np.std(lens)
    y_err_1 = np.percentile(lens, 80) - mean
    y_err_2 = mean - np.percentile(lens, 20)
    
    length_dists[label] = SimpleNamespace(mean=mean, std=std, y_err_1=y_err_1, y_err_2=y_err_2)

outputs = [length_dists[lab] for lab in red_labels]
means = np.array([i.mean for i in outputs])
errs  = np.array([[i.y_err_1, i.y_err_2] for i in outputs]).T

print(errs)
fig, ax = plt.subplots(figsize=(15, 8))
ax.bar(red_labels, means, yerr=errs, align='center', alpha=0.5, ecolor='black', capsize=10)
plt.ylabel('average number of words', size=16)

### Finding Biased Words

In [ ]:
def flatten(x):
    return [i for j in x for i in j]

### Get vocabulary of words in more than 0.5% of examples
all_utts = [utt for utts in utts_per_label.values() for utt in utts]
num_utts = len(all_utts)
vocab_thresh = num_utts/100

num_ex_with_word = Counter(flatten([list(set(utt)) for utt in all_utts])) #num examples with word
vocab = [word for word, count in num_ex_with_word.most_common() if count > vocab_thresh]
vocab_to_id = {word: k for k, word in enumerate(vocab)}
id_to_vocab = {v:k for k, v in vocab_to_id.items()}

### Find label occurences per word. i.e if hello = [0.2, 0, 0.8], then 20% of hello in data set 1, 80% in label 3
label_word_counter = {}
for DA_label, utts in utts_per_label.items():
    word_counts = Counter(flatten(utts))
    label_word_counter[DA_label] = word_counts

vocab_label_dist = {}
for word in vocab:
    label_counts = [label_word_counter[lab][word] for lab in all_labels]
    v_label_dist = np.array(label_counts)/sum(label_counts)
    vocab_label_dist[word] = v_label_dist

In [ ]:
print(label_dist)

In [ ]:
import seaborn as sns

### get indexes of words with highest margin
word_vecs = np.array([vocab_label_dist[word] for word in vocab])[:,:9]
max_1 = np.max(word_vecs, axis=-1)
word_index = np.argsort(max_1)[::-1]

### now get words, and their word distribution
words = [id_to_vocab[k] for k in word_index[:20]]
dists = np.array([word_vecs[k] for k in word_index[:20]])
freqs = np.array([100*num_ex_with_word[word]/num_utts for word in words])

### Do a heatplot for visualisation
fig, axes = plt.subplots(1, 2, figsize=(10,10), gridspec_kw={'width_ratios': [2, 1]})

sns.heatmap(dists, annot=True, cmap='Blues', fmt='.2f', 
            xticklabels=red_labels, yticklabels=words, 
           ax=axes[0], cbar=False)
axes[0].patch.set_edgecolor('black')  
axes[0].patch.set_linewidth('2')  

sns.barplot(freqs, words, orient='h', ax=axes[1])
axes[1].set_xlabel('% of utterances with given word')

In [ ]:
import seaborn as sns

### get indexes of words with highest margin
norm_label_dist = np.array(label_dist)/sum(label_dist)
word_vecs = np.array([vocab_label_dist[word]/norm_label_dist for word in vocab])[:,:9]
max_1 = np.max(word_vecs, axis=-1)
word_index = np.argsort(max_1)[::-1]

### now get words, and their word distribution
words = [id_to_vocab[k] for k in word_index[:20]]
dists = np.array([word_vecs[k] for k in word_index[:20]])
freqs = np.array([100*num_ex_with_word[word]/num_utts for word in words])

### Do a heatplot for visualisation
fig, axes = plt.subplots(1, 2, figsize=(12,10), gridspec_kw={'width_ratios': [2, 1]})

sns.heatmap(dists, annot=True, cmap='Blues', fmt='.2f', 
            xticklabels=red_labels, yticklabels=words, 
           ax=axes[0], cbar=False)
axes[0].patch.set_edgecolor('black')  
axes[0].patch.set_linewidth('2')

sns.barplot(freqs, words, orient='h', ax=axes[1])
axes[1].set_xlabel('% of utterances with given word')

In [ ]:
sns.barplot(all_labels[:13], norm_label_dist[:13])
plt.xticks(rotation=70)
plt.ylabel('P(y)')

In [ ]:
"""searches through all vectors to find the ones with largest margin between max and second max"""

### Find word distributions for each label i.e if vocab = ['the', 'a'], then [0.6, 0.1] means 60% of words of current label is the and 10% is a
data_set_word_dist = {}
for DA_label, utts in utts_per_label.items():
    DA_vocab_vec = np.zeros(len(vocab))
    word_counts = Counter(flatten(utts))
    word_dist = [(word, count/len(flatten(utts))) for word, count in word_counts.items()]
    
    for word, fraction in word_dist:
        if word in vocab:
            word_id = vocab_to_id[word]
            DA_vocab_vec[word_id] = fraction
    data_set_word_dist[DA_label] = DA_vocab_vec

word_vecs = np.array([output_vecs[label] for label in red_label_list])
word_vecs = word_vecs.T

max_1 = np.max(word_vecs, axis=-1)
max_2 = np.sort(word_vecs, axis=-1)[:,-2]
word_index = np.argsort(max_1-max_2)[::-1]


print(red_label_list)
for k in word_index[:10]:
    print(id_to_vocab[k])
    print(word_vecs[k])
    
#print((max_1-max_2)[word_index])
#print(word_vecs.T[word_index])
#a = word_vecs.T[ind_1] - word_vecs.T[ind_2]
#print(a.shape)

In [ ]:
print(word_vecs.T[ind_1].shape)

In [ ]:
def flatten(x):
    return [i for j in x for i in j]

### Get vocabulary of words in more than 0.5% of examples
all_utts = [utt for utts in utts_per_label.values() for utt in utts]
all_words = flatten([list(set(utt)) for utt in all_utts])
vocab_thresh = len(all_utts)/200
print(vocab_thresh)
vocab_count = Counter(all_words)
vocab = [word for word, count in vocab_count.most_common() if count > vocab_thresh]
vocab_to_id = {word: k for k, word in enumerate(vocab)}
id_to_vocab = {v:k for k, v in vocab_to_id.items()}


In [ ]:
import time

for conv in train_data:
    for utt in conv:
        print(utt)
        time.sleep(2)